In [1]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Headless Chrome setup
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

url = "https://aiche.confex.com/aiche/2023/meetingapp.cgi/Program/3307"
driver.get(url)

# Wait for the calendar content to load (wait for any CalendarList)
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.CLASS_NAME, "CalendarList"))
)

soup = BeautifulSoup(driver.page_source, "html.parser")

ul_calendar = soup.find("ul", class_="Calendar")

data = []

if ul_calendar:
    calendar_lists = ul_calendar.find_all("ul", class_="CalendarList")

    for cal_list in calendar_lists:
        # Extract date (first li span with class defaultTZ inside .date span)
        date_li = cal_list.find("li")
        date = None
        if date_li:
            date_span = date_li.find("span", class_="defaultTZ")
            if date_span:
                date = date_span.get_text(strip=True)

        # Extract time from <time class="first">
        time_tag = cal_list.find("time", class_="first")
        time_range = time_tag.get_text(strip=True) if time_tag else None

        # Extract sessions inside <section class="itemCalendar ...">
        sessions = []
        session_sections = cal_list.find_all("section", class_="itemCalendar")
        for session in session_sections:
            # Session title and link
            a_tag = session.find("a")
            session_title = a_tag.get_text(strip=True) if a_tag else None
            session_link = a_tag['href'] if a_tag and 'href' in a_tag.attrs else None

            # Speakers - bold tags inside span with class topDisplay
            speakers = []
            top_display = session.find("span", class_="topDisplay")
            if top_display:
                bolds = top_display.find_all("b")
                for b in bolds:
                    speakers.append(b.get_text(strip=True))

            # Location info inside ul.propertyInfo > li.propertyName
            location = None
            prop_info = session.find("ul", class_="propertyInfo")
            if prop_info:
                prop_name = prop_info.find("li", class_="propertyName")
                if prop_name:
                    location = prop_name.get_text(strip=True)

            sessions.append({
                "title": session_title,
                "link": session_link,
                "speakers": speakers,
                "location": location
            })

        data.append({
            "date": date,
            "time": time_range,
            "sessions": sessions
        })

else:
    print("❌ <ul class='Calendar'> not found.")

driver.quit()

# Save JSON data to file
with open("sessions.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("✅ Data saved to sessions.json")


✅ Data saved to sessions.json


In [ ]:
# import json
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# import time

# BASE_URL = "https://aiche.confex.com/aiche/2023/meetingapp.cgi/"

# def get_first_session_url(json_file_path):
#     with open(json_file_path, "r", encoding="utf-8") as file:
#         sessions_data = json.load(file)

#     for entry in sessions_data:
#         for session in entry.get("sessions", []):
#             session_link = session.get("link")
#             if session_link:
#                 return BASE_URL + session_link
#     return None


# def scrape_details_section(url):
#     options = Options()
#     options.add_argument("--headless")
#     driver = None

#     try:
#         driver = webdriver.Chrome(options=options)
#         print(f"🔗 Scraping: {url}")
#         driver.get(url)

#         WebDriverWait(driver, 20).until(
#             EC.presence_of_element_located((By.TAG_NAME, "body"))
#         )

#         # Save full page source to debug
#         html = driver.page_source
#         with open("debug_page.html", "w", encoding="utf-8") as f:
#             f.write(html)
#         print("✅ Page source saved to debug_page.html")

#         # Try to extract Details section
#         details_section = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CSS_SELECTOR, "section#Details"))
#         )
#         print("✅ Extracted Details Section:\n")
#         print(details_section.get_attribute("outerHTML"))

#     except Exception as e:
#         print(f"❌ Error loading or locating section: {e}")

#     finally:
#         if driver:
#             driver.quit()


# # --- Run everything ---
# first_url = get_first_session_url("sessions.json")
# if first_url:
#     scrape_details_section(first_url)
# else:
#     print("No session URL found.")


🔗 Scraping: https://aiche.confex.com/aiche/2023/meetingapp.cgi/Session/52334
✅ Page source saved to debug_page.html
❌ Error loading or locating section: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff78f47fea5+79173]
	GetHandleVerifier [0x0x7ff78f47ff00+79264]
	(No symbol) [0x0x7ff78f239e5a]
	(No symbol) [0x0x7ff78f290586]
	(No symbol) [0x0x7ff78f29083c]
	(No symbol) [0x0x7ff78f2e4247]
	(No symbol) [0x0x7ff78f2b89af]
	(No symbol) [0x0x7ff78f2e100d]
	(No symbol) [0x0x7ff78f2b8743]
	(No symbol) [0x0x7ff78f2814c1]
	(No symbol) [0x0x7ff78f282253]
	GetHandleVerifier [0x0x7ff78f74a2dd+3004797]
	GetHandleVerifier [0x0x7ff78f74472d+2981325]
	GetHandleVerifier [0x0x7ff78f763380+3107360]
	GetHandleVerifier [0x0x7ff78f49aa2e+188622]
	GetHandleVerifier [0x0x7ff78f4a22bf+219487]
	GetHandleVerifier [0x0x7ff78f488df4+115860]
	GetHandleVerifier [0x0x7ff78f488fa9+116297]
	GetHandleVerifier [0x0x7ff78f46f558+11256]
	BaseThreadInitThunk [0x0x7ff81425e8d7+23]
	RtlUserThreadStart [0x0x7ff81635c34c+44]



In [2]:
import json
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

BASE_URL = "https://aiche.confex.com/aiche/2023/meetingapp.cgi/"
all_links_by_session = {}

def get_all_session_urls(json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as file:
        sessions_data = json.load(file)

    session_urls = []
    for entry in sessions_data:
        for session in entry.get("sessions", []):
            session_link = session.get("link")
            if session_link:
                session_urls.append(BASE_URL + session_link)

    return session_urls


def extract_presentation_links_from_live_page(url):
    options = Options()
    options.add_argument("--headless")
    driver = None

    try:
        driver = webdriver.Chrome(options=options)
        print(f"🔗 Opening URL: {url}")
        driver.get(url)

        # Wait for Presentations section to load

        WebDriverWait(driver, 40).until_not(
                    EC.text_to_be_present_in_element(
                        (By.TAG_NAME, "body"),
                        "please wait while the program loads"
                    )
                )

        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "section.field_ChildList_PaperSlot"))
        )
        time.sleep(5)
        print("✅ Presentations section found.")

        # Additional wait to allow dynamic content to load inside the <ul>
        print("⏳ Waiting 10 seconds for dynamic content inside <ul> to load...")
        time.sleep(10)

        # Wait for at least one <a> inside the <ul.PaperSlot> if possible
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul.PaperSlot a[href^='Paper/']"))
        )
        print("✅ At least one paper link found inside <ul.PaperSlot>.")

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        presentation_section = soup.select_one("section.field_ChildList_PaperSlot")
        if not presentation_section:
            print("❌ Presentations section not found after waiting.")
            return []

        presentation_section_ul = presentation_section.find("ul", class_="PaperSlot")
        if not presentation_section_ul:
            print("❌ <ul> with class PaperSlot not found.")
            return []

        links = []
        for a_tag in presentation_section_ul.find_all("a", href=True):
            href = a_tag["href"]
            if href.startswith("Paper/"):
                full_link = BASE_URL + href
                links.append(full_link)

        return links

    except Exception as e:
        print(f"❌ Error: {e}")
        return []

    finally:
        if driver:
            driver.quit()



# --- Run everything ---
def main():
    all_session_urls = get_all_session_urls("sessions.json")
    all_links_by_session = {}

    if not all_session_urls:
        print("❌ No session URLs found.")
        return

    for idx, session_url in enumerate(all_session_urls, start=1):
        print(f"\n🔍 Processing session {idx} / {len(all_session_urls)}: {session_url}")
        links = extract_presentation_links_from_live_page(session_url)
        print(f"✅ Found {len(links)} presentation links.")
        all_links_by_session[session_url] = links

    # Save results to JSON
    output_file = "presentation_links_by_session_3330.json"
    output_dir = os.path.dirname(output_file)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(all_links_by_session, f, indent=2)

    print(f"\n✅ All presentation links saved to {output_file}")

if __name__ == "__main__":
    main()


🔍 Processing session 1 / 59: https://aiche.confex.com/aiche/2023/meetingapp.cgi/Session/52066
🔗 Opening URL: https://aiche.confex.com/aiche/2023/meetingapp.cgi/Session/52066
✅ Presentations section found.
⏳ Waiting 10 seconds for dynamic content inside <ul> to load...
✅ At least one paper link found inside <ul.PaperSlot>.
✅ Found 7 presentation links.

🔍 Processing session 2 / 59: https://aiche.confex.com/aiche/2023/meetingapp.cgi/Session/52097
🔗 Opening URL: https://aiche.confex.com/aiche/2023/meetingapp.cgi/Session/52097
✅ Presentations section found.
⏳ Waiting 10 seconds for dynamic content inside <ul> to load...
✅ At least one paper link found inside <ul.PaperSlot>.
✅ Found 8 presentation links.

🔍 Processing session 3 / 59: https://aiche.confex.com/aiche/2023/meetingapp.cgi/Session/52096
🔗 Opening URL: https://aiche.confex.com/aiche/2023/meetingapp.cgi/Session/52096
✅ Presentations section found.
⏳ Waiting 10 seconds for dynamic content inside <ul> to load...
✅ At least one paper

In [5]:
import json
import os
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Setup headless browser ---
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

def get_text(soup, selector):
    elements = soup.select(selector)
    if not elements:
        return "Not found"
    return " | ".join([el.get_text(strip=True) for el in elements])

def load_all_links(json_file_path):
    if not os.path.exists(json_file_path):
        print(f"❌ File {json_file_path} not found.")
        return []

    with open(json_file_path, "r", encoding="utf-8") as f:
        session_links = json.load(f)

    all_links = []
    for session_url, links in session_links.items():
        all_links.extend(links)
    print(f"✅ Loaded {len(all_links)} total presentation links.")
    return all_links

def extract_and_save_presentation_data(links, output_file="aiche_papers.json"):
    if not links:
        print("⚠️ No links to process.")
        return

    driver = webdriver.Chrome(options=options)
    all_data = []

    try:
        for idx, url in enumerate(links, start=1):
            print(f"\n🔍 Processing ({idx}/{len(links)}): {url}")
            try:
                driver.get(url)
                WebDriverWait(driver, 20).until(
                    EC.any_of(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "section.titleContent")),
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.field_Abstract"))
                    )
                )
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                data = {
                    "url": url,
                    "topic": get_text(soup, "p.favoriteItem"),
                    "date_time": get_text(soup, 'span.defaultTZ'),
                    "abstract": get_text(soup, 'section.field_Abstract'),
                    "presenting_author": f"{get_text(soup, 'a.presenter')} | {get_text(soup, 'span.roleAffiliation')}"
                }
                all_data.append(data)
                print("✅ Extracted successfully.")
                print("📝 Abstract Preview:")
                print("\n".join(data["abstract"].splitlines()[:2]))  # show 2 lines max
            except Exception as e:
                print(f"❌ Failed to extract from {url}\n   Error: {e}")

    finally:
        driver.quit()

    # Ensure output file creation
    output_dir = os.path.dirname(output_file)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_data, f, ensure_ascii=False, indent=2)

    print(f"\n✅ All data saved to {output_file}")

# --- Main Execution ---
if __name__ == "__main__":
    links = load_all_links("presentation_links_by_session.json")
    extract_and_save_presentation_data(links, "aiche_papers_.json")


✅ Loaded 153 total presentation links.

🔍 Processing (1/153): https://aiche.confex.com/aiche/2023/meetingapp.cgi/Paper/668173
✅ Extracted successfully.
📝 Abstract Preview:
AbstractIn recent years, machine learning (ML) methodshave transformed computational chemistry and materials research. At the start of any machine learning inquiry we must first ask: how do we represent our system? This question is at the core of many ML methods, ranging from end-to-end methods, that take "raw" data formats and use multi-layer architectures to implicitly encode the chemical information to "feature-forward" methods, that use physics and chemistry to design numerical representations conducive to machine learning workflows. While both approaches have merits and overlaps, the latter often allows for added tunability to identify the role of key parameters in the design space and more straightforward interpretation.There are many ways to encode the raw chemical data into features, and the suitable choice l